In [1]:
import numpy as np
import unicodedata
import pandas as pd
import string
punctuation = string.punctuation
import nltk
import re
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer # converts text data into numerical format
import re
stop_words = set(stopwords.words("english"))

def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ''.join(list_text_new)

import string
punctuation = string.punctuation
def punctuation_removal(messy_str):
    clean_list = [char for char in messy_str if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

# Create a function to remove special characters
def remove_special_characters(text):
    pat = r'[^a-zA-z0-9]'
    return re.sub(pat, ' ', text)


def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def remove_stopwords(sentence):
    words = sentence.split()  # Tokenize the sentence into words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# function for stemming
def get_stem(text):
    stemmer = nltk.porter.PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bysan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bysan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd

In [4]:
from bertopic import BERTopic

In [5]:
topic_model_pos =  BERTopic(embedding_model = "all-MiniLM-L6-v2")
topic_model_neg =  BERTopic(embedding_model = "all-MiniLM-L6-v2")

In [6]:
df2 =  pd.read_excel('sentiment_scores.xlsx')

In [8]:
df2['sentiment'] = df2['sentiment_score'].apply(lambda x: x[2:10])

In [27]:
df_pos = df2[df2['sentiment'] == 'POSITIVE']

In [33]:
df_neg = df2[df2['sentiment'] == 'NEGATIVE']

In [29]:
df_pos = df_pos.dropna(subset=['_text'])

In [34]:
df_neg = df_neg.dropna(subset=['_text'])

In [40]:
df_pos['_text'] = df_pos['_text'].apply(drop_numbers)

In [38]:
df_neg['_text'] = df_neg['_text'].apply(drop_numbers)

In [41]:
df_pos['_text'] = df_pos['_text'].astype(str)
df_neg['_text'] = df_neg['_text'].astype(str)

In [ ]:
topics_neg,probs_neg = topic_model_neg.fit_transform(df_neg['_text'])

In [ ]:
topics_pos,probs_pos = topic_model_pos.fit_transform(df_pos['_text'])

In [ ]:
df_Pos_res = topic_model_pos.get_topic_info()

In [ ]:
df_Neg_res = topic_model_pos.get_topic_info()

In [ ]:
topic_to_docs_pos = {}
for doc_idx, topic in enumerate(topics_pos):
    if topic not in topic_to_docs_pos:
        topic_to_docs[topic] = []
    topic_to_docs[topic].append(doc_idx)

In [ ]:
topic_to_docs_neg = {}
for doc_idx, topic in enumerate(topics_neg):
    if topic not in topic_to_docs_neg:
        topic_to_docs[topic] = []
    topic_to_docs[topic].append(doc_idx)

In [ ]:
df_pos['topic'] = None
for key, value in topic_to_docs.items():
    df_pos.loc[value, 'topic'] = key+1

In [ ]:
df_neg['topic'] = None
for key, value in topic_to_docs.items():
    df_neg.loc[value, 'topic'] = key+1

In [152]:
df_pos['_text'] = df_pos['_text'].fillna('').astype(str)
grouped_reviews_pos = df_pos.groupby('title')['_text'].apply(' '.join).reset_index()

In [159]:
df_neg['_text'] = df_neg['_text'].fillna('').astype(str)
grouped_reviews_neg = df_neg.groupby('title')['_text'].apply(' '.join).reset_index()

In [154]:
topics_pos,probs_pos = topic_model_pos.fit_transform(grouped_reviews_pos['_text'])

In [160]:
topics_neg,probs_neg = topic_model_neg.fit_transform(grouped_reviews_neg['_text'])

In [156]:
topic_model_pos.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11,-1_pizza_great_food_good,"[pizza, great, food, good, service, italian, p...",[peyton matt amazing genuine want best custome...
1,0,392,0_food_great_good_service,"[food, great, good, service, place, delicious,...",[delicious food comfortable price grilled toas...
2,1,68,1_food_great_good_service,"[food, great, good, service, mexican, taco, pl...",[toddler love place staff really sweet appreci...
3,2,21,2_pizza_great_good_service,"[pizza, great, good, service, place, best, tim...",[absolutely love place atmosphere welcoming fo...
4,3,18,3_italian_food_great_service,"[italian, food, great, service, pasta, restaur...",[honestly layered flavor pasta dish tried incr...


In [161]:
topic_model_neg.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9,-1_food_back_restaurant_time,"[food, back, restaurant, time, like, order, it...",[italian need say muy rico todo cant go wrong ...
1,0,470,0_food_order_time_service,"[food, order, time, service, back, place, like...",[chili reduced portion size considerably far c...
2,1,31,1_pizza_order_time_ordered,"[pizza, order, time, ordered, place, one, deli...",[every time order pizza really good get order ...


In [162]:
import pandas as pd
from bertopic import BERTopic
from sklearn.model_selection import ParameterGrid

texts = df_pos['_text'].tolist()

param_grid = {
    'n_gram_range': [(1, 1), (1, 2), (1, 3)],
    'min_topic_size': [5, 10, 15],
    'nr_topics': ["auto", 5, 10, 15]
}

best_score = float('-inf')
best_model = None
best_params = None

for params in ParameterGrid(param_grid):
    model = BERTopic(**params)
    topics, _ = model.fit_transform(texts)
    score = model.get_topic_info()['Count'].sum()
    if score > best_score:
        best_score = score
        best_model = model
        best_params = params

print("Best Parameters:", best_params)

df_pos['dominant_topic'] = best_model.transform(df_pos['_text'])[0]

dominant_topic_per_restaurant = df_pos.groupby('title')['dominant_topic'].agg(lambda x: x.value_counts().index[0])

print(dominant_topic_per_restaurant)

KeyboardInterrupt: 

In [170]:
import pandas as pd
from bertopic import BERTopic

texts = df_neg['_text'].tolist()

model = BERTopic(embedding_model="all-MiniLM-L6-v2")

topics, _ = model.fit_transform(texts)

df_neg['dominant_topic'] = topics
df_neg['dominant_topic_name'] = df_neg['dominant_topic'].apply(lambda x: topic_names[topic_names['Topic'] == x]['Name'].values[0] if x != -1 else 'No dominant topic')

dominant_topic_per_restaurant = df_neg.groupby('title')['dominant_topic'].agg(lambda x: x.value_counts().index[0])

print(dominant_topic_per_restaurant)


KeyboardInterrupt: 

In [171]:
import pandas as pd
from bertopic import BERTopic

texts = df_neg['_text'].tolist()

# Initialize and fit BERTopic model
model = BERTopic(embedding_model="all-MiniLM-L6-v2")
topics, _ = model.fit_transform(texts)

# Add dominant topics to the dataframe
df_neg['dominant_topic'] = topics

# Extract topic names/descriptions
topic_names = model.get_topic_info()

# Map dominant topic indices to topic names
df_neg['dominant_topic_name'] = df_neg['dominant_topic'].apply(
    lambda x: topic_names[topic_names['Topic'] == x]['Name'].values[0] 
    if x != -1 else 'No dominant topic'
)

# Group by restaurant and get the most frequent dominant topic
dominant_topic_per_restaurant = df_neg.groupby('title')['dominant_topic_name'].agg(
    lambda x: x.value_counts().index[0]
)

print(dominant_topic_per_restaurant)


title
3 Amigos Mexican Bar & Grill                                            No dominant topic
5Church Buckhead                                                    228_peer_1205_122_225
AG                                         1659_estaban_servicefooddrinks_midtier_osteria
ANNUNZIATA'S Pizza & Pasta                                              No dominant topic
AZN Sandwich Bar                                                        No dominant topic
                                                                ...                      
Yume Ramen Bar in Peachtree City                                        No dominant topic
Zafron Restaurant                                                       No dominant topic
Zaxby's Chicken Fingers & Buffalo Wings                                 No dominant topic
il Giallo Osteria & Bar                             2065_potstickers_ravioli_nowhere_near
la Madeleine                                                            No dominant topic
Name

In [167]:
topic_to_docs = {}
for doc_idx, topic in enumerate(topics):
    if topic not in topic_to_docs:
        topic_to_docs[topic] = []
    topic_to_docs[topic].append(doc_idx)

In [172]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7465,-1_la_moes_que_los,"[la, moes, que, los, great, amazing, de, littl...",[si eres mexicano solamente te tratan bien si ...
1,0,163,0_postmates_mistake_blame_sabzi,"[postmates, mistake, blame, sabzi, toolet, thr...",[horrible experience frita ’ tonight phone dis...
2,1,121,1_muy_buena_comida_deliciosa,"[muy, buena, comida, deliciosa, recomiendo, to...","[muy buena la comida, muy buena la comida, muy..."
3,2,120,2_bang_deplorable_salmon_michelin,"[bang, deplorable, salmon, michelin, sunk, can...",[patroned ny eve first let start limited 90 mi...
4,3,117,3_coffe_marlows_coffee_choking,"[coffe, marlows, coffee, choking, ordera, 1152...",[unpleasant experience came coffe ghosted barm...
...,...,...,...,...,...
2221,2220,10,2220_pleasantry_obligatory_blandegg_warmcold,"[pleasantry, obligatory, blandegg, warmcold, m...",[today first time staff used obligatory pleasa...
2222,2221,10,2221_boiling_workday_drinkable_carry,"[boiling, workday, drinkable, carry, soup, oth...",[terrible experience ordered noodle soup lunch...
2223,2222,10,2222_10th_screw_dairy_four,"[10th, screw, dairy, four, finish, allergy, co...",[10th time food wrong dairy allergy cheese fry...
2224,2223,10,2223_slower_messed_decent_waiter,"[slower, messed, decent, waiter, disorganized,...","[never slower waiter decent food messed order,..."
